## Recommendation : Open on Kaggle

This notebook provides a comparison between the clip, dino and full (clip+dino) efficiency on image classification

In [1]:
!git clone https://github.com/thaid27/fairness_project.git
!mv fairness_project/* .
!mv fairness_project/.* . 2>/dev/null  # Pour les fichiers cachés comme .gitignore (ignore les erreurs)
!rm -rf fairness_project

Cloning into 'fairness_project'...
remote: Enumerating objects: 63, done.
remote: Total 63 (delta 0), reused 0 (delta 0), pack-reused 63 (from 1)
Receiving objects: 100% (63/63), 62.87 MiB | 38.00 MiB/s, done.
Resolving deltas: 100% (21/21), done.
mv: cannot move 'fairness_project/example_images' to './example_images': Directory not empty
mv: cannot move 'fairness_project/utils' to './utils': Directory not empty


In [2]:
!pip install -q open_clip_torch
!pip install -q --upgrade optimum
!pip install -q bitsandbytes
!pip install -q transformers==4.46.3

!pip install -q segment_anything
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

  Preparing metadata (setup.py) ... done


In [3]:
from pathlib import Path
import os, shutil, random, json, math, cv2, torch, numpy as np, matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm


# CONFIG
DATASET_ROOT = Path("/kaggle/working")
OUTPUT_ROOT = Path("/kaggle/working/nyu_output")
OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

zip_path_nyu = "/kaggle/working/nyu.zip"
zip_path_hardparse = "/kaggle/working/hardparse.zip"

# Dézipper le contenu dans DATASET_ROOT
import zipfile
with zipfile.ZipFile(zip_path_nyu, 'r') as zip_ref:
    zip_ref.extractall(DATASET_ROOT)

with zipfile.ZipFile(zip_path_hardparse, 'r') as zip_ref:
    zip_ref.extractall(DATASET_ROOT)


INPAINT_PROMPT = "A realistic and seamless continuation of the background, with no objects — just an empty, natural extension of the scene."
OBJECT_DETECTION_PROMPT = 'Mark all the objects in this scene. Return only the coordinates of the objects.'
NUM_INPAINTING_PER_OBJECT = 1

device0 = torch.device("cuda:0")
device1 = torch.device("cuda:1")

In [4]:
torch.cuda.empty_cache()

In [6]:
import torch
import torchvision.transforms as T
import numpy as np
import cv2
from PIL import Image
from tqdm import tqdm
from utils.inpainting import inpainting_for_scoring, inpainting_remove_object
import csv
import random

from transformers import CLIPProcessor, CLIPModel, AutoModelForCausalLM, AutoProcessor
import timm
from segment_anything import SamPredictor, sam_model_registry
from diffusers import DiffusionPipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

2025-06-17 12:41:04.856994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750164064.890229     246 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750164064.899070     246 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [7]:
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device0).eval()
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

dino_model = timm.create_model('vit_small_patch8_224_dino', pretrained=True).to(device0)
dino_model.eval()

NUM_INPAINTING_PER_OBJECT = 1  # Ajustable
pipe_stable_diffusion = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting").to(device1)

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.46G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [13]:
def find_file(folder, basename, exts=("jpg", "jpeg", "png", "webp")):
    for ext in exts:
        path = os.path.join(folder, f"{basename}.{ext}")
        if os.path.isfile(path):
            return path
    raise FileNotFoundError(f"❌ {basename} with extensions {exts} not found in {folder}")

def load_triplet(folder):
    """Return PIL images: rgb, mask_A (1st), mask_B (2nd), auto-detecting extensions."""
    rgb_path   = find_file(folder, "img")
    maskA_path = find_file(folder, "A")
    maskB_path = find_file(folder, "B")

    rgb   = Image.open(rgb_path).convert("RGB")
    maskA = Image.open(maskA_path).convert("L")
    maskB = Image.open(maskB_path).convert("L")

    return rgb, maskA, maskB

def combined_score(inpaint_imgs, orig_img, clip_model, clip_proc, dino_model, mode="full"):
    scores = []

    if mode in ("full", "clip_only"):
        imgs = clip_proc(images=[orig_img] + inpaint_imgs, return_tensors="pt").to(device0)
        with torch.no_grad():
            emb = clip_model.get_image_features(**imgs)
            emb = torch.nn.functional.normalize(emb, dim=1)
            sims = (emb[1:] @ emb[0].unsqueeze(0).T).squeeze().detach().cpu().numpy()
        scores.append(sims)

    if mode in ("full", "dino_only"):
        preprocess = T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
        ])
        tt = torch.stack([preprocess(im) for im in [orig_img] + inpaint_imgs]).to(device0)
        with torch.no_grad():
            emb = dino_model(tt)
            emb = torch.nn.functional.normalize(emb, dim=1)
            sims = (emb[1:] @ emb[0].unsqueeze(0).T).squeeze().detach().cpu().numpy()
        scores.append(sims)

    return np.mean(np.stack(scores), axis=0).mean()

def pipeline_choice_btw_2_masks(image_path, mode="full", verbose=False):
    """
    mode ∈ {"full", "clip_only", "dino_only"}
    """
    image, maskA, maskB = load_triplet(image_path)
    masks = [maskA, maskB]
    scores = []

    for mask in masks:
        inpainted_images = []
        for _ in range(NUM_INPAINTING_PER_OBJECT):
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (13, 13))
            mask_np = np.array(mask)
            dilated_mask = cv2.dilate(mask_np, kernel, iterations=1)
            mask_PIL = Image.fromarray(dilated_mask).convert("L").resize((512, 512))

            inpainted_image = inpainting_for_scoring(image, mask_PIL, pipe_stable_diffusion)
            inpainted_images.append(inpainted_image)

        div = combined_score(
            inpainted_images, image,
            clip_model, clip_processor,
            dino_model, mode=mode
        )
        scores.append(div)

    idx_max = int(np.argmax(scores))
    return idx_max == 0

In [ ]:
dataset = 'hardparse'

if dataset == 'nyu':
    nyu_dir = "nyu/"

    file_list = sorted([f for f in os.listdir(nyu_dir) if f != ".DS_Store"])

    subset_size = max(1, len(file_list) // 10)
    file_subset = random.sample(file_list, subset_size)

    print(f"Processing {subset_size} files out of {len(file_list)} total files (10% of dataset)") # Adjust as needed
    
if dataset == 'hardparse':
    nyu_dir = "hardparse/"

    file_subset = sorted([f for f in os.listdir(nyu_dir) if f != ".DS_Store"])

In [ ]:
methods = {
    "Ours_full":     "full",
    "Ours_clip":     "clip_only",
    "Ours_dino":     "dino_only",
}

stats = {k:0 for k in methods}
total  = len(file_subset)
i=1
for fname in file_subset:
    print(f"{i}/{total}")
    i += 1
    fpath = os.path.join(nyu_dir, fname)
    print(fpath)
    gt_is_A_first = True
    for name, mode in methods.items():
        print(name)
        pred = pipeline_choice_btw_2_masks(fpath, mode=mode, verbose=False)
        if pred == gt_is_A_first:
            stats[name] += 1

Processing 4 files out of 40 total files (10% of dataset)
1/40
hardparse/01
Ours_full


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Ours_clip


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Ours_dino


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
csv_path = f"ablation_clip_dino_{dataset}.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["method", "correct", "total", "accuracy (%)"])
    for k,v in stats.items():
        w.writerow([k, v, total, f"{100*v/total:.2f}"])
print(f"Résultats sauvegardés dans {csv_path}")

Résultats sauvegardés dans ablation_clip_dino_hardparse.csv
